In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import time
import math

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def name_call(day, order): 
    directory = './Mapmatching_output/18-04-'+format(day, '02')+'/'
    filename = 'speed'+'_'+format(day, '02')+'_'+format(order, '03')+'.pkl'
    full_name = directory + filename
    return full_name

In [ ]:
datelist = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27] ## Consider only weekdays (Mon–Fri) for the month of April.
datalength = 144 * len(datelist)
print(f'Dataset contains total {len(datelist)} days, and the length is {datalength}.')

In [ ]:
def get_time_list(num):
    time_list = [num + 144*i for i in range(-40, 40)]
    valid_list = [value for value in time_list if value>= 0 and value < datalength and value!=num]
    return valid_list

In [ ]:
edge_number = 451 # You should input the exact nodes number of the study area network 
col_list = list(range(0, edge_number))
Speed = pd.DataFrame(columns = col_list)

start = time.time()

for i in datelist:
    for j in range(0, 144):
        filename = name_call(i, j)
        try:
            data = pd.read_pickle(filename)
            Speed = Speed.append(data.T.loc['Speed'], ignore_index=True)

        except:
            empty_data = pd.Series(dtype='float')
            Speed = Speed.append(empty_data, ignore_index=True)
            
print(f'Total processing time: {time.time()-start} seconds.')

In [ ]:
start = time.time()

for i in range(0, edge_number):
    for j in range(0, datalength):
        try:
            if math.isnan(Speed[j][i]) == True: 
                if math.isnan(Speed[j+1][i]) == False:
                    if ((Speed[j-1][i] + Speed[j+1][i]) / 2) >= 0:
                        Speed[j][i] = (Speed[j-1][i] + Speed[j+1][i]) / 2
                    else:
                        raise NotImplementedError
                else:
                    other_days_list = pd.Series([Speed[k][i] for k in get_time_list(j)])
                    if other_days_list.mean() >= 0:
                        Speed[j][i] = other_days_list.mean()
                    else:
                        raise NotImplementedError
            else:
                continue

        except : ## Interpolate using the overall mean at this time slot
            sample = Speed[j]
            sample2 = sample[(~np.isnan(sample)) & (sample > 0)]
            Speed[j][i] = sample2.mean()
            
print(f'Total processing time: {time.time()-start} seconds.')

In [ ]:
for i in range(edge_number):
    if Speed[i].isnull().sum() > 0:
        print(f'#{i} Link should be observed. The length of null data is {Speed[i].isnull().sum()}')

## If the above interpolation and correction process worked correctly, 
## the expected output here should be 0 (i.e., no missing values remaining).

In [ ]:
Speed.to_csv('../../input_data/speed_data.csv', index=False)